First, the DICOM format images were converted to NRRD format.

In [ ]:
import SimpleITK as sitk
import os

In [ ]:
def dicom_series_to_nrrd(input_folder, output_nrrd):
    dicom_names = sitk.ImageSeriesReader.GetGDCMSeriesFileNames(input_folder)  
    reader = sitk.ImageSeriesReader()
    reader.SetFileNames(dicom_names) 
    dicom_series = reader.Execute() 
    sitk.WriteImage(dicom_series, output_nrrd)

In [ ]:
path = "/"

for root, dirs, files in os.walk(path):
    for dir_name in dirs:
        output_file = "/"+ dir_name +".nrrd"
        dicom_series_to_nrrd(os.path.join(root, dir_name), output_file)

Next, N4 bias field correction was performed.

In [ ]:
import SimpleITK as sitk
import glob
import os

In [ ]:
path =  '/' 
outputpath = '/'

In [ ]:
for file_abs in glob.glob(path):
    imagePath = file_abs
    input_image = sitk.ReadImage(imagePath)
    mask_image = sitk.OtsuThreshold(input_image,0,1,200)
    input_image = sitk.Cast(input_image, sitk.sitkFloat32)
    corrector = sitk.N4BiasFieldCorrectionImageFilter()
    output_image = corrector.Execute(input_image,mask_image)
    output_image = sitk.Cast(output_image, sitk.sitkInt16)
    (filepath, tempfilename) = os.path.split(file_abs)
    (filename, extension) = os.path.splitext(tempfilename)
    tempfilename = filename + "_N4" + extension
    file_out = os.path.join(outputpath, tempfilename)
    sitk.WriteImage(output_image, file_out)
        
print('Completed!')

The NRRD format images and VOIs were resampled to a voxel size of 1×1×1 mm, followed by normalization of the resulting data.

In [ ]:
import SimpleITK as sitk
import numpy as np
import glob
import os

In [ ]:
def resample_image(itk_image, out_spacing=None):
    if out_spacing is None:
        out_spacing = [1.0, 1.0, 1.0]
    original_spacing = itk_image.GetSpacing()
    original_size = itk_image.GetSize()

    out_size = [
        int(np.round(original_size[0] * original_spacing[0] / out_spacing[0])),
        int(np.round(original_size[1] * original_spacing[1] / out_spacing[1])),
        int(np.round(original_size[2] * original_spacing[2] / out_spacing[2]))
    ]

    resample = sitk.ResampleImageFilter()
    resample.SetOutputSpacing(out_spacing)
    resample.SetSize(out_size)
    resample.SetOutputDirection(itk_image.GetDirection())
    resample.SetOutputOrigin(itk_image.GetOrigin())
    resample.SetTransform(sitk.Transform())
    resample.SetDefaultPixelValue(itk_image.GetPixelIDValue())

    resample.SetInterpolator(sitk.sitkBSpline)

    return resample.Execute(itk_image)

def resample_mask(itk_mask, out_spacing=None):
    if out_spacing is None:
        out_spacing = [1.0, 1.0, 1.0]
    original_spacing = itk_mask.GetSpacing()
    original_size = itk_mask.GetSize()

    out_size = [
        int(np.round(original_size[0] * original_spacing[0] / out_spacing[0])),
        int(np.round(original_size[1] * original_spacing[1] / out_spacing[1])),
        int(np.round(original_size[2] * original_spacing[2] / out_spacing[2]))
    ]

    resample = sitk.ResampleImageFilter()
    resample.SetOutputSpacing(out_spacing)
    resample.SetSize(out_size)
    resample.SetOutputDirection(itk_mask.GetDirection())
    resample.SetOutputOrigin(itk_mask.GetOrigin())
    resample.SetTransform(sitk.Transform())
    resample.SetDefaultPixelValue(itk_mask.GetPixelIDValue())

    resample.SetInterpolator(sitk.sitkNearestNeighbor)

    return resample.Execute(itk_mask)

#Normalization
sitk_NIF = sitk.NormalizeImageFilter ()

Radiomic features were extracted using PyRadiomics, and the data were standardized using Z-score normalization.

In [ ]:
import pandas as pd
import numpy as np
import radiomics
import glob
import os

from radiomics import featureextractor

In [ ]:
extractor=featureextractor.RadiomicsFeatureExtractor("exampleMR_NoResampling.yaml") #The bincount was set to 64 bins.

In [ ]:
imagepath = '/'
maskpath = '/'

In [ ]:
data={}
dfs=[]
rank = ['pre','ap','pvp','delayed','fs'] #Different phase
for phase in rank:
    for image,mask in zip(glob.glob(imagepath),glob.glob(maskpath)):
        if image.endswith(phase + '_N4_resamplezscore.nrrd'):
            try:
                feature=extractor.execute(image,mask) 
            except:
                print("There is something wrong with this :" , image)
            featurepd=pd.DataFrame([feature])
            featurepd.rename(columns=lambda x: x + '_'+ phase , inplace=True)
            if image.endswith('pre_N4_resamplezscore.nrrd'):
                (filepath, tempfilename) = os.path.split(image)
                featurepd.insert(0,"id",tempfilename.split('_')[0])           
        else:
            featurepd = pd.DataFrame(data)
        dfs.append(featurepd)
    radiomic = pd.concat(dfs)
    dfs=[]
    # radiomic = radiomic.iloc[:,23:]
    if phase == "pre":
        radiomics = pd.concat([radiomic.iloc[:,0],radiomic.iloc[:,23:]],axis=1)
    else:
        radiomics = pd.concat([radiomics,radiomic.iloc[:,22:]],axis=1)

# radiomics

In [ ]:
radiomicsfeatures = radiomics.iloc[:,1:]

In [ ]:
from sklearn.preprocessing import StandardScaler

In [ ]:
scaler = StandardScaler()
radiomicsfeatures_zscore = scaler.fit_transform(radiomicsfeatures)
radiomicsfeaturesafterzscore = pd.DataFrame(radiomicsfeatures_zscore)

Feature selection: 

An initial consistency test was performed to select features with intra- and inter-class correlation coefficients greater than 0.75.

In [ ]:
import pingouin as pg
import pandas as pd
import numpy as np
import os

In [ ]:
data_1.insert(0,"reader",np.ones(data_1.shape[0]))
data_2.insert(0,"reader",np.ones(data_2.shape[0])*2)

In [ ]:
data = pd.concat([data_1,data_2]) # make a data frame like the test data 
columns_list = data.columns.tolist()
featuresname = columns_list[2:]
len(featuresname)

In [ ]:
threshold = 0.75
icc_resultzunei2 = []
scores = []
for i in featuresname:
    icc = pg.intraclass_corr(data = data, targets = "target", raters = "reader",ratings = i)
    score = icc.loc[2,'ICC']
    if score > threshold:
        icc_resultzunei2.append(i)
        #scores.append(score)
    else:
        continue    

In [ ]:
len(icc_resultzunei2)

In [ ]:
listzujian=icc_resultzujian

In [ ]:
listzunei1=icc_resultzunei1

In [ ]:
listzunei2=icc_resultzunei2

In [ ]:
#Features with intra-class correlation coefficients greater than 0.75 in both tests and an inter-class correlation coefficient greater than 0.75 were selected, and their intersection was taken.
intersection = set(listzujian).intersection(listzunei1, listzunei2)
selectedfeatures = intersection
len(selectedfeatures)

In [ ]:
dataall = pd.read_csv('/')
dfall = pd.DataFrame(dataall)

icc_df = dfall.filter(selectedfeatures)

In [ ]:
def get_all_files_in_folder(folder_path):
    return os.listdir(folder_path)
def get_first_part_of_string(string_list):
    return [string.split("_")[0] for string in string_list]

In [ ]:
#插入id列

name_path = 'E:/preprocessing/shundeyiyuan/1/20min'
file_names1= get_all_files_in_folder(name_path)
a = get_first_part_of_string(file_names1)
# b = a[::6]
icc_df.insert(loc=0, column='id', value=a)

In [ ]:
icc_df.to_csv("E:/preprocessing/shundeyiyuan/2/extract/outcome/resultaftericc075.csv")
#手动插入label

logistic＞0.05

In [ ]:
import statsmodels.api as sm
import pandas as pd
from collections import Counter

In [ ]:
Counter(y == 1)

In [ ]:
single_result = pd.DataFrame()
for i in range(0, X.shape[1], 1):
    x = X.iloc[:, i]
    model = sm.Logit(y, sm.add_constant(x))
    results = model.fit()
    coef_df = pd.DataFrame({"params": results.params,  
                            "std err": results.bse,  
                            "t": round(results.tvalues, 3),  
                            "p-values": round(results.pvalues, 3)  
                            })
    coef_df[['coef_0.025', 'coef_0.975']] = results.conf_int()  
    single_result = pd.concat([single_result, coef_df.drop(labels='const')])

Pearson＞0.75

In [ ]:
def delcol_corr_new1(data, method='pearson', threshold=0.8):
    df = pd.DataFrame(data)
    correlation_matrix = df.corr(method).abs()
    correlation_matrix = correlation_matrix.where(np.triu(np.ones(correlation_matrix.shape), k=1).astype(bool))
    correlation_matrix = correlation_matrix.fillna(0)
    selected_variables = []
    for i in correlation_matrix.columns:
        for j in correlation_matrix[i].index:
            if correlation_matrix.loc[i, j] > threshold:
                if correlation_matrix.loc[i].drop([i, j]).mean() > correlation_matrix.loc[j].drop([i, j]).mean():
                    colname = i
                    if colname not in selected_variables:
                        selected_variables.append(colname)
                else:
                    colname = j
                    if colname not in selected_variables:
                        selected_variables.append(colname)
    df = df.drop(selected_variables,axis=1)
    return df

Lasso

In [ ]:
from sklearn.linear_model import LassoCV
import numpy as np

In [ ]:
alpharange = np.logspace(-10,0,100, base=10) 

In [ ]:
lasso_ = LassoCV(alphas=alpharange 
                ,cv=10 
                ).fit(Xtrain, Ytrain)

XGBoost

In [ ]:
from sklearn.datasets import make_classification
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from collections import Counter
from imblearn.over_sampling import SMOTE
import numpy as np
import xgboost as xgb
import matplotlib.pyplot as plt
from xgboost import XGBClassifier as XGBC
from sklearn.datasets import make_blobs
from sklearn.model_selection import train_test_split as TTS
from sklearn.metrics import confusion_matrix as cm, recall_score as recall, roc_auc_score as auc

In [ ]:
datatrain = pd.read_csv('/',index_col=0)
ytrain = datatrain.loc[:,'label']
Xtrain = datatrain.drop(['label'], axis=1)

In [ ]:
import xgboost as xgb
from xgboost import XGBRegressor as XGBR
from sklearn.ensemble import RandomForestRegressor as RFR
from sklearn.linear_model import LinearRegression as LinearR
from sklearn.datasets import load_boston
from sklearn.model_selection import KFold, cross_val_score as CVS ,train_test_split as TTS
from sklearn.metrics import mean_squared_error as MSE

import pandas as pd
import numpy as np
import matplotlib.pyplot as  plt
from time import time
import datetime

from collections import Counter

import xgboost as xgb
from time import time
import datetime

In [ ]:
for i in [0.01,0.05,0.1,0.15,0.3,0.6,0.8,1,3,6,8,10,15,20,25,30]:
    clf8 = XGBC(n_estimators=8,scale_pos_weight=i).fit(Xtrain,ytrain)
    ypred_ = clf8.predict(X_vali)
    print(i)
    print("\tAccuracy:{}".format(clf8.score(X_vali,y_vali)))
    print("\tRecall:{}".format(recall(y_vali,ypred_)))
    print("\tAUC:{}".format(auc(y_vali,clf8.predict_proba(X_vali)[:,1])))

scale_pos_weight=sum(negative instances)/sum(positive instances)

In [ ]:
from sklearn.model_selection import KFold, cross_val_score as CVS ,train_test_split as TTS
from sklearn.metrics import mean_squared_error as MSE
# import sklearn
# sorted(sklearn.metrics.SCORERS.keys())
from sklearn import metrics
import matplotlib.pyplot as plt

In [ ]:
clf = XGBC(n_estimators=10,
               subsample=0.27,
               learning_rate=0.4,
               scale_pos_weight=0.17
               ).fit(Xtrain,ytrain)
print(auc(ytrain,clf.predict_proba(Xtrain)[:,1]))
print(auc(y_vali,clf.predict_proba(X_vali)[:,1]))
print(auc(y_vali2,clf.predict_proba(X_vali2)[:,1]))

Save the model

In [ ]:
import pickle

In [ ]:
pickle.dump(clf, open("xgboostforVETCMTM.dat","wb"))

In [ ]:
clf = pickle.load(open("xgboostforVETC.dat", "rb"))

In [ ]:
ypredt = loaded_model.predict_proba(Xtrain)[:,1]
radscoretrain=pd.concat([datatrain,pd.DataFrame(ypredt)],axis=1)
radscoretrain.to_csv('')

In [ ]:
ypredv = loaded_model.predict_proba(X_vali)[:,1]
radscorevali=pd.concat([vali_data,pd.DataFrame(ypredv)],axis=1)
radscorevali.to_csv("")

In [ ]:
ypredv2 = loaded_model.predict_proba(X_vali2)[:,1]
radscorevali2=pd.concat([vali2_data,pd.DataFrame(ypredv2)],axis=1)
radscorevali2.to_csv("")